In [1]:
import pandas as pd

# Cargar CSV
df = pd.read_csv('../data/raw/propiedades.csv')

# Inspeccionar columnas para confirmar nombres exactos
print(df.columns)

# Elimina espacios en nombres de columnas por seguridad
df.columns = df.columns.str.strip()

# Si las coordenadas vienen como texto, conviértelas a float
df['Latitud'] = pd.to_numeric(df['Latitud'], errors='coerce')
df['Longitud'] = pd.to_numeric(df['Longitud'], errors='coerce')

# Rango geográfico estimado para la Región Metropolitana
lat_min, lat_max = -34, -32
lon_min, lon_max = -71, -69

# Verificar coordenadas fuera del rango válido
coordenadas_invalidas = df[
    (df['Latitud'] < lat_min) | (df['Latitud'] > lat_max) |
    (df['Longitud'] < lon_min) | (df['Longitud'] > lon_max)
]

print(f"Coordenadas inválidas encontradas: {len(coordenadas_invalidas)}")

# Mostrar registros inválidos
coordenadas_invalidas[['Comuna', 'Latitud', 'Longitud']].head()


Index(['Comuna', 'Link', 'Tipo_Vivienda', 'N_Habitaciones', 'N_Baños',
       'N_Estacionamientos', 'Total_Superficie', 'Superficie_Construida',
       'Valor_UF', 'Valor_CLP', 'Dirección', 'Quien_Vende', 'Corredor',
       'Latitud', 'Longitud', 'Descripcion', 'Unnamed: 16'],
      dtype='object')
Coordenadas inválidas encontradas: 127


,Comuna,Latitud,Longitud
119,Melipilla,-33.858611,-71.233611
162,Melipilla,-33.570563,-71.212040
170,Melipilla,-33.678842,-71.194469
306,Curacaví,-33.357648,-71.150990
542,La Reina,-35.675147,-71.542969


In [2]:
# Filtrar solo propiedades dentro del rango válido
df_validas = df[
    (df['Latitud'] >= lat_min) & (df['Latitud'] <= lat_max) &
    (df['Longitud'] >= lon_min) & (df['Longitud'] <= lon_max)
]

# Guardar resultado limpio
df_validas.to_csv('../data/processed/propiedades_rm.csv', index=False)


In [3]:
import folium

# Crear mapa centrado en Santiago
mapa = folium.Map(location=[-33.45, -70.65], zoom_start=11)

# Agregar puntos al mapa
for _, row in df_validas.iterrows():
    folium.CircleMarker(
        location=[row['Latitud'], row['Longitud']],
        radius=2,
        color='blue',
        fill=True,
        fill_opacity=0.6
    ).add_to(mapa)

# Guardar como HTML para abrirlo en tu navegador
mapa.save('../reports/mapa_propiedades_validas.html')


In [7]:
import pandas as pd

# Cargar dataset original o geolocalizado previamente
ruta_entrada = "../data/processed/propiedades_rm.csv"
ruta_salida = "../data/processed/propiedades_geolocalizadas.csv"

try:
    df = pd.read_csv(ruta_salida)
    print("Se cargó archivo previamente guardado.")
except FileNotFoundError:
    df = pd.read_csv(ruta_entrada)
    df['Comuna_real'] = None  # Inicializar columna

# Asegurar columna ID
if 'ID' not in df.columns:
    df = df.reset_index().rename(columns={"index": "ID"})

df.head(3)


,ID,Comuna,Link,Tipo_Vivienda,N_Habitaciones,N_Baños,N_Estacionamientos,Total_Superficie,Superficie_Construida,Valor_UF,Valor_CLP,Dirección,Quien_Vende,Corredor,Latitud,Longitud,Descripcion,Unnamed: 16,Comuna_real
0,0,Conchalí,https://chilepropiedades.cl/ver-publicacion/ve...,Casa,3,2,3,160.0,0.0,2929.64,115000000,Sandra 1437,Urbanet propiedades,Urbanet,-33.381808,-70.667314,Amplia casa esquina ubicada en tranquilo secto...,NaN,None
1,1,Lampa,https://chilepropiedades.cl/ver-publicacion/ve...,Casa,4,1,2,78.0,50.0,1732.31,68000000,Los Alerces,Casol Propiedades,Casol Propiedades,-33.286489,-70.882639,"Ubicación: Pasaje Los Alerces, Villa Santa Ros...",NaN,None
2,2,Padre Hurtado,https://chilepropiedades.cl/ver-publicacion/ve...,Casa,3,1,3,70.0,45.0,2165.38,85000000,Av. San Ignacio 1,Claudia,Dueño Directo,-33.581050,-70.810799,Araya Propiedades VENDE acogedora casa de dos ...,NaN,None


In [8]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Inicializar geolocalizador
geolocator = Nominatim(user_agent="propiedades-santiago")
geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)

# Función para obtener comuna
def obtener_comuna_real(lat, lon):
    try:
        location = geocode((lat, lon), language='es')
        if location and 'address' in location.raw:
            return location.raw['address'].get('suburb') or \
                   location.raw['address'].get('city_district') or \
                   location.raw['address'].get('town') or \
                   location.raw['address'].get('city') or \
                   location.raw['address'].get('municipality')
    except:
        return None


In [9]:
for i, row in df.iterrows():
    if pd.isna(row['Comuna_real']):
        comuna_obtenida = obtener_comuna_real(row['Latitud'], row['Longitud'])
        df.at[i, 'Comuna_real'] = comuna_obtenida
        print(f"[{i+1}/{len(df)}] ID={row['ID']} → {comuna_obtenida}")
    
    if i % 100 == 0 and i != 0:
        df.to_csv(ruta_salida, index=False)
        print(f"> Progreso guardado hasta fila {i}")

print("✅ Geolocalización completa (o reanudada)")


[1/9850] ID=0 → Conchalí
[2/9850] ID=1 → Lampa
[3/9850] ID=2 → Padre Hurtado
[4/9850] ID=3 → La Florida
[5/9850] ID=4 → Lampa
[6/9850] ID=5 → Puente Alto
[7/9850] ID=6 → Chicureo
[8/9850] ID=7 → Calera de Tango
[9/9850] ID=8 → Ñuñoa
[10/9850] ID=9 → Maipú
[11/9850] ID=10 → Peñalolén
[12/9850] ID=11 → San Bernardo
[13/9850] ID=12 → Maipú
[14/9850] ID=13 → Maipú
[15/9850] ID=14 → Santiago
[16/9850] ID=15 → Recoleta
[17/9850] ID=16 → Maipú
[18/9850] ID=17 → La Florida
[19/9850] ID=18 → Huechuraba
[20/9850] ID=19 → Ñuñoa
[21/9850] ID=20 → Puente Alto
[22/9850] ID=21 → San Bernardo
[23/9850] ID=22 → Maipú
[24/9850] ID=23 → Maipú
[25/9850] ID=24 → La Florida
[26/9850] ID=25 → Puente Alto
[27/9850] ID=26 → La Florida
[28/9850] ID=27 → El Bosque
[29/9850] ID=28 → Ñuñoa
[30/9850] ID=29 → Maipú
[31/9850] ID=30 → Lo Prado
[32/9850] ID=31 → Cerrillos
[33/9850] ID=32 → Puente Alto
[34/9850] ID=33 → Estación Central
[35/9850] ID=34 → Puente Alto
[36/9850] ID=35 → Puente Alto
[37/9850] ID=36 → Batuco

In [23]:
# Mostrar diferencias antes de aplicar correcciones
df['Comuna_match_raw'] = df['Comuna'] == df['Comuna_real']

diferencias_raw = df[df['Comuna_match_raw'] == False]

print("🔍 Cantidad de diferencias sin corrección:", len(diferencias_raw))
diferencias_raw[['ID', 'Comuna', 'Comuna_real', 'Latitud', 'Longitud']].head(20)


🔍 Cantidad de diferencias sin corrección: 791


,ID,Comuna,Comuna_real,Latitud,Longitud
6,6,Colina,Chicureo,-33.276525,-70.630183
19,19,Providencia,Ñuñoa,-33.437386,-70.582164
36,36,Lampa,Batuco,-33.229694,-70.810469
76,76,Til til,Tiltil,-33.131256,-70.866184
94,94,San Bernardo,None,-33.583333,-70.700000
98,98,Huechuraba,Recoleta,-33.380430,-70.646820
99,99,Colina,Chicureo,-33.265764,-70.623881
100,100,Lampa,Batuco,-33.229694,-70.810469
106,106,Quilicura,None,-33.361229,-70.709159
108,108,Puente Alto,Santiago,-33.591479,-70.542948


In [41]:
# Crea tu propio diccionario después de analizar
correcciones_comunas = {
    "Chicureo": "Colina",
    "Batuco": "Lampa",
    "Chicureo Oriente": "Colina",
    "Tiltil": "Til til",
    "Linderos": "Buin",
    "Malloco": "Peñaflor",
    "Valle Grande": "Lampa",
    "Alto Jahuel": "Buin",
    "Esmeralda": "Colina"
}


In [42]:

# Mostrar todas las diferencias
display(diferencias[['ID', 'Comuna', 'Comuna_real', 'Comuna_real_corregida', 'Latitud', 'Longitud']])

# Aplicar correcciones de comuna
df['Comuna_real_corregida'] = df['Comuna_real'].map(correcciones_comunas).fillna(df['Comuna_real'])

# Comparar comuna del scraping con la corregida
df['Comuna_match'] = df['Comuna'] == df['Comuna_real_corregida']

# Filtrar las filas con diferencias
diferencias = df[df['Comuna_match'] == False]

# Mostrar resumen
print(f"🔍 Total de diferencias encontradas tras aplicar correcciones: {len(diferencias)}")

# Mostrar todas las diferencias (puedes usar .head() o .sample() si son muchas)
diferencias[['ID', 'Comuna', 'Comuna_real', 'Comuna_real_corregida', 'Latitud', 'Longitud']]


,ID,Comuna,Comuna_real,Comuna_real_corregida,Latitud,Longitud
19,19,Providencia,Ñuñoa,Ñuñoa,-33.437386,-70.582164
94,94,San Bernardo,None,None,-33.583333,-70.700000
98,98,Huechuraba,Recoleta,Recoleta,-33.380430,-70.646820
106,106,Quilicura,None,None,-33.361229,-70.709159
108,108,Puente Alto,Santiago,Santiago,-33.591479,-70.542948
208,208,Las Condes,La Reina,La Reina,-33.452034,-70.538507
214,214,La Pintana,La Granja,La Granja,-33.557674,-70.621498
268,268,Santiago,Estación Central,Estación Central,-33.445062,-70.679508
302,302,Peñalolén,La Florida,La Florida,-33.512169,-70.550747
322,322,Isla de Maipo,Talagante,Talagante,-33.708378,-70.865164


🔍 Total de diferencias encontradas tras aplicar correcciones: 379


,ID,Comuna,Comuna_real,Comuna_real_corregida,Latitud,Longitud
19,19,Providencia,Ñuñoa,Ñuñoa,-33.437386,-70.582164
94,94,San Bernardo,None,None,-33.583333,-70.700000
98,98,Huechuraba,Recoleta,Recoleta,-33.380430,-70.646820
106,106,Quilicura,None,None,-33.361229,-70.709159
108,108,Puente Alto,Santiago,Santiago,-33.591479,-70.542948
208,208,Las Condes,La Reina,La Reina,-33.452034,-70.538507
214,214,La Pintana,La Granja,La Granja,-33.557674,-70.621498
268,268,Santiago,Estación Central,Estación Central,-33.445062,-70.679508
302,302,Peñalolén,La Florida,La Florida,-33.512169,-70.550747
322,322,Isla de Maipo,Talagante,Talagante,-33.708378,-70.865164


In [45]:
import numpy as np

# 1. Reemplazar NaN en Comuna_real_corregida por "Quilicura" solo donde Comuna == "Quilicura"
cond_quilicura_nan = (df['Comuna'] == "Quilicura") & (df['Comuna_real_corregida'].isna())
df.loc[cond_quilicura_nan, 'Comuna_real_corregida'] = "Quilicura"

# 2. Ahora hacer match con valores que pueden tener NaN correctamente
df['Comuna_match'] = df['Comuna'] == df['Comuna_real_corregida']

# 3. Filtrar registros con diferencia
diferencias_finales = df[df['Comuna_match'] == False]
cantidad_dropeados = len(diferencias_finales)

# Mostrar registros a eliminar
print(f"❌ Total registros a eliminar: {cantidad_dropeados}")
display(diferencias_finales[['ID', 'Comuna', 'Comuna_real', 'Comuna_real_corregida', 'Latitud', 'Longitud']])

# 4. Filtrar registros que sí coinciden (mantener)
df_limpio = df[df['Comuna_match'] == True].copy()

# 5. Reemplazar columna original "Comuna" con la corregida
df_limpio['Comuna'] = df_limpio['Comuna_real_corregida']

# 6. Eliminar columnas auxiliares
df_limpio = df_limpio.drop(columns=[
    col for col in ['Comuna_real', 'Comuna_real_corregida', 'Comuna_match'] if col in df_limpio.columns
])

# 7. Guardar CSV limpio
ruta_salida = "../data/processed/propiedades_geolocalizadas_final.csv"
df_limpio.to_csv(ruta_salida, index=False)

print(f"✅ Archivo guardado en: {ruta_salida}")
print(f"📊 Registros totales después de limpieza: {len(df_limpio)}")
print(f"❌ Registros eliminados por diferencia de comuna: {cantidad_dropeados}")


❌ Total registros a eliminar: 241


,ID,Comuna,Comuna_real,Comuna_real_corregida,Latitud,Longitud
19,19,Providencia,Ñuñoa,Ñuñoa,-33.437386,-70.582164
94,94,San Bernardo,None,None,-33.583333,-70.700000
98,98,Huechuraba,Recoleta,Recoleta,-33.380430,-70.646820
108,108,Puente Alto,Santiago,Santiago,-33.591479,-70.542948
208,208,Las Condes,La Reina,La Reina,-33.452034,-70.538507
214,214,La Pintana,La Granja,La Granja,-33.557674,-70.621498
268,268,Santiago,Estación Central,Estación Central,-33.445062,-70.679508
302,302,Peñalolén,La Florida,La Florida,-33.512169,-70.550747
322,322,Isla de Maipo,Talagante,Talagante,-33.708378,-70.865164
353,353,La Florida,Santiago,Santiago,-33.474606,-70.651157


✅ Archivo guardado en: ../data/processed/propiedades_geolocalizadas_final.csv
📊 Registros totales después de limpieza: 9609
❌ Registros eliminados por diferencia de comuna: 241


In [46]:
# Mostrar registros que serán eliminados por diferencia en comuna
print(f"❌ Total registros a eliminar: {cantidad_dropeados}")
display(diferencias_finales[['ID', 'Comuna', 'Comuna_real', 'Comuna_real_corregida', 'Latitud', 'Longitud']])


❌ Total registros a eliminar: 241


,ID,Comuna,Comuna_real,Comuna_real_corregida,Latitud,Longitud
19,19,Providencia,Ñuñoa,Ñuñoa,-33.437386,-70.582164
94,94,San Bernardo,None,None,-33.583333,-70.700000
98,98,Huechuraba,Recoleta,Recoleta,-33.380430,-70.646820
108,108,Puente Alto,Santiago,Santiago,-33.591479,-70.542948
208,208,Las Condes,La Reina,La Reina,-33.452034,-70.538507
214,214,La Pintana,La Granja,La Granja,-33.557674,-70.621498
268,268,Santiago,Estación Central,Estación Central,-33.445062,-70.679508
302,302,Peñalolén,La Florida,La Florida,-33.512169,-70.550747
322,322,Isla de Maipo,Talagante,Talagante,-33.708378,-70.865164
353,353,La Florida,Santiago,Santiago,-33.474606,-70.651157


In [47]:
import numpy as np

# Reemplazar NaN en Comuna_real_corregida por "Lampa" solo donde Comuna == "Lampa"
cond_lampa_nan = (df['Comuna'] == "Lampa") & (df['Comuna_real_corregida'].isna())
df.loc[cond_lampa_nan, 'Comuna_real_corregida'] = "Lampa"

# Recalcular coincidencia
df['Comuna_match'] = df['Comuna'] == df['Comuna_real_corregida']

# Filtrar registros con diferencia
diferencias_finales = df[df['Comuna_match'] == False]
cantidad_dropeados = len(diferencias_finales)

# Mostrar registros eliminados
print(f"❌ Total registros a eliminar: {cantidad_dropeados}")
display(diferencias_finales[['ID', 'Comuna', 'Comuna_real', 'Comuna_real_corregida', 'Latitud', 'Longitud']])

# Filtrar registros que coinciden
df_limpio = df[df['Comuna_match'] == True].copy()

# Actualizar columna 'Comuna' con la corregida
df_limpio['Comuna'] = df_limpio['Comuna_real_corregida']

# Eliminar columnas auxiliares
df_limpio = df_limpio.drop(columns=[
    col for col in ['Comuna_real', 'Comuna_real_corregida', 'Comuna_match'] if col in df_limpio.columns
])

# Guardar CSV limpio
ruta_salida = "../data/processed/propiedades_geolocalizadas_final.csv"
df_limpio.to_csv(ruta_salida, index=False)

print(f"✅ Archivo guardado en: {ruta_salida}")
print(f"📊 Registros totales después de limpieza: {len(df_limpio)}")
print(f"❌ Registros eliminados por diferencia de comuna: {cantidad_dropeados}")


❌ Total registros a eliminar: 196


,ID,Comuna,Comuna_real,Comuna_real_corregida,Latitud,Longitud
19,19,Providencia,Ñuñoa,Ñuñoa,-33.437386,-70.582164
94,94,San Bernardo,None,None,-33.583333,-70.700000
98,98,Huechuraba,Recoleta,Recoleta,-33.380430,-70.646820
108,108,Puente Alto,Santiago,Santiago,-33.591479,-70.542948
208,208,Las Condes,La Reina,La Reina,-33.452034,-70.538507
214,214,La Pintana,La Granja,La Granja,-33.557674,-70.621498
268,268,Santiago,Estación Central,Estación Central,-33.445062,-70.679508
302,302,Peñalolén,La Florida,La Florida,-33.512169,-70.550747
322,322,Isla de Maipo,Talagante,Talagante,-33.708378,-70.865164
353,353,La Florida,Santiago,Santiago,-33.474606,-70.651157


✅ Archivo guardado en: ../data/processed/propiedades_geolocalizadas_final.csv
📊 Registros totales después de limpieza: 9654
❌ Registros eliminados por diferencia de comuna: 196
